#pytorch teangan sorangan

In [ ]:
%pip install transformers
%pip install scipy

In [6]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
import json 
from scipy.special import softmax 
from decimal import Decimal

In [2]:
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
def analyzeSentiment(text):
    text = preprocess(text)
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    object_sentiment = {
        "positive": float(scores[2]),
        "neutral": float(scores[1]),
        "negative": float(scores[0])
    }
    return object_sentiment

In [10]:
#testing
text = "I love you"
sentiment = analyzeSentiment(text)
print(sentiment)

{'positive': 0.8594690561294556, 'neutral': 0.12239762395620346, 'negative': 0.01813327521085739}


In [11]:
json_post = json.load(open('photos_updated_withComent.json'))
#add sentiment to json

for post in json_post:
    for comment in post['comments']:
        comment['sentiment'] = analyzeSentiment(comment['comment_text'])

with open('photos_updated_withComent_sentiment.json', 'w') as outfile:
    json.dump(json_post, outfile)